## Import required libraries
### Author: Sameer
### Date: May 2019

In [4]:
import numpy as np
import scipy as sp
from numpy.linalg import LinAlgError
import os
from datetime import datetime
from copy import deepcopy

## Parameters Dictionary

In [ ]:
class gen_param_dict(object):
    '''
    Init function not required. Currently there for debugging.
        init: define the class constructor of with no arguments
    '''
    def __init__(self, verbose=True):
        self.verbose = verbose
        if self.verbose:
            print('-'*25)
            print('Call to the class gen_param_dict has started')
            print('-'*25)
        
    def gen_param_ALG(self, inner_iterations=1, min_eta=1e-5, kl_step=0.2, min_step_mult=0.01,
                     max_step_mult=10.0, min_mult=0.1, max_mult=5.0, initial_state_var=1e-6, 
                     init_traj_distr=None, traj_opt=None, max_ent_traj=0.0, dynamics=None, 
                     cost=None, sample_on_policy=False, fit_dynamics=True):
        # Algorithm
        ALG = {
            # Number of iterations.
            'inner_iterations': inner_iterations,  
            # Minimum initial lagrange multiplier in DGD for trajectory optimization.
            'min_eta': min_eta,        
            'kl_step': kl_step,
            'min_step_mult': min_step_mult,
            'max_step_mult': max_step_mult,
            'min_mult': min_mult,
            'max_mult': max_mult,
            # Trajectory settings.
            'initial_state_var': initial_state_var,
            # A list of initial LinearGaussianPolicy objects for each condition.
            'init_traj_distr': init_traj_distr, 
            # Trajectory optimization.
            'traj_opt': traj_opt,
            # Weight of maximum entropy term in trajectory optimization.
            'max_ent_traj': max_ent_traj,
            # Dynamics hyperaparams.
            'dynamics': dynamics,
            # Costs.
            'cost': cost,  # A list of Cost objects for each condition.
            # Whether or not to sample with neural net policy (only for badmm/mdgps).
            'sample_on_policy': sample_on_policy,
            # Inidicates if the algorithm requires fitting of the dynamics.
            'fit_dynamics': fit_dynamics,    
        }
        return ALG
    
    def gen_param_ALG_BADMM(self, inner_iterations=4.0, policy_dual_rate=0.1, policy_dual_rate_covar=0.0,
                           fixed_lg_step=0.0, lg_step_schedule=10.0, ent_reg_schedule=0.0, init_pol_wt=0.01,
                           policy_sample_mode='add', exp_step_increase=2.0, exp_step_decrease=0.5, 
                            exp_step_upper=0.5, exp_step_lower=1.0): 
        # Algorithm BADMM
        ALG_BADMM = {
            'inner_iterations': inner_iterations,
            'policy_dual_rate': policy_dual_rate,
            'policy_dual_rate_covar': policy_dual_rate_covar,
            'fixed_lg_step': fixed_lg_step,
            'lg_step_schedule': lg_step_schedule,
            'ent_reg_schedule': ent_reg_schedule,
            'init_pol_wt': init_pol_wt,
            'policy_sample_mode': policy_sample_mode,
            'exp_step_increase': exp_step_increase,
            'exp_step_decrease': exp_step_decrease,
            'exp_step_upper': exp_step_upper,
            'exp_step_lower': exp_step_lower,
        }
        return ALG_BADMM
    
    def gen_param_TRAJ_OPT_LQR(self, del0=1e-4, eta_error_threshold=1e16, min_eta=1e-8, max_eta=1e16,
                              cons_per_step=False, use_prev_distr=False, update_in_bwd_pass=True):
        TRAJ_OPT_LQR = {
            'del0': del0,
            'eta_error_threshold': eta_error_threshold,
            'min_eta': min_eta,
            'max_eta': max_eta,
            # Whether or not to enforce separate KL constraints at each time step.
            'cons_per_step': cons_per_step,  
            # Whether or not to measure expected KL under the previous traj distr.
            'use_prev_distr': use_prev_distr,  
            # Whether or not to update the TVLG controller during the bwd pass.
            'update_in_bwd_pass': update_in_bwd_pass,  
        }
        return TRAJ_OPT_LQR
    
    def gen_param_INIT_LG_LQR(self, init_var=1.0, stiffness=1.0, stiffness_vel=0.5,
                             final_weight=1.0):
        # Initial Linear Gaussian Trajectory distribution, LQR-based initializer.
        INIT_LG_LQR = {
            'init_var': init_var,
            'stiffness': stiffness,
            'stiffness_vel': stiffness_vel,
            'final_weight': final_weight,
            # Parameters for guessing dynamics
            # dU vector of accelerations, default zeros.
            'init_acc': [],  
            # dU vector of gains, default ones.
            'init_gains': [],  
        }
        return INIT_LG_LQR
    
    def gen_param_POLICY_PRIOR(self, strength=1e-4):
        # PolicyPrior
        POLICY_PRIOR = {
            'strength': strength,
        }
        return POLICY_PRIOR
    
    def gen_param_POLICY_PRIOR_GMM(self, min_samples_per_cluster=20.0, max_clusters=50.0, max_samples=20.0,
                                  strength=1.0):
        # PolicyPriorGMM
        POLICY_PRIOR_GMM = {
            'min_samples_per_cluster': min_samples_per_cluster,
            'max_clusters': max_clusters,
            'max_samples': max_samples,
            'strength': strength,
        }
        return POLICY_PRIOR_GMM
    
    def gen_param_COST_STATE(self, RAMP_CONSTANT=1.0, l1=0.0, l2=1.0, alpha=1e-2, wp_final_multiplier=1.0, 
                            target_state=None, wp=None):
        # CostState
        COST_STATE = {
            # How target cost ramps over time.
            'ramp_option': RAMP_CONSTANT,  
            'l1': l1,
            'l2': l2,
            'alpha': alpha,
            # Weight multiplier on final time step.
            'wp_final_multiplier': 1.0,  
            'data_types': {
                'JointAngle': {
                    # Target state - must be set.
                    'target_state': target_state,  
                    # State weights - must be set.
                    'wp': wp,  
                },
            },
        }
        return COST_STATE
    
    def gen_param_COST_SUM(self, costs=[], weights=[]):
        # CostSum
        COST_SUM = {
            # A list of hyperparam dictionaries for each cost.
            'costs': costs,  
            # Weight multipliers for each cost.
            'weights': weights,  
        }
        return COST_SUM
    
    def gen_param_COST_ACTION(self, wu=np.array([])):
        # CostAction
        COST_ACTION = {
            # Torque penalties, must be 1 x dU numpy array.
            'wu': np.array([]),  
        }
        return COST_ACTION
        


## Required class with different functionalities 

### Class General Utilities

In [ ]:
class general_utlis(object):
    '''
    Init function not required. Currently there for debugging.
        init: define the class constructor of with no arguments
    '''
    def __init__(self, verbose=True):
        self.verbose = verbose
        if self.verbose:
            print('-'*25)
            print('Call to the class general_utlis has started')
            print('-'*25)
    
    def check_shape(self, value, expected_shape, name=''):
    """
    Throws a ValueError if value.shape != expected_shape.
    Args:
        value: Matrix to shape check.
        expected_shape: A tuple or list of integers.
        name: An optional name to add to the exception message.
    """
    if value.shape != tuple(expected_shape):
        raise ValueError('Shape mismatch %s: Expected %s, got %s' % (name, str(expected_shape), str(value.shape)))
    
    def extract_condition(self, hyperparams, m):
    """
    Pull the relevant hyperparameters corresponding to the specified
    condition, and return a new hyperparameter dictionary.
    Simple explanation:
        This function does the following:
            If given dictionary it takes all keys and values. And makes new dicitonary out of it.
            Additionally if one of the values is a list then it takes one element out specified by
            the user through the parameter "m"
    """
    return {var: val[m] if isinstance(val, list) else val for var, val in hyperparams.items()}

    def approx_equal(self, a, b, threshold=1e-5):
    """
    Return whether two numbers are equal within an absolute threshold.
    Returns:
        True if a and b are equal within threshold.
    """
    return np.all(np.abs(a - b) < threshold)

    def finite_differences(self, func, inputs, func_output_shape=(), epsilon=1e-5):
    """
    Computes gradients via finite differences.
    derivative = (func(x+epsilon) - func(x-epsilon)) / (2*epsilon)
    Args:
        func: Function to compute gradient of. Inputs and outputs can be
            arbitrary dimension.
        inputs: Vector value to compute gradient at.
        func_output_shape: Shape of the output of func. Default is
            empty-tuple, which works for scalar-valued functions.
        epsilon: Difference to use for computing gradient.
    Returns:
        Gradient vector of each dimension of func with respect to each
        dimension of input.
    """
    gradient = np.zeros(inputs.shape+func_output_shape)
    for idx, _ in np.ndenumerate(inputs):
        test_input = np.copy(inputs)
        test_input[idx] += epsilon
        obj_d1 = func(test_input)
        assert obj_d1.shape == func_output_shape
        test_input = np.copy(inputs)
        test_input[idx] -= epsilon
        obj_d2 = func(test_input)
        assert obj_d2.shape == func_output_shape
        diff = (obj_d1 - obj_d2) / (2 * epsilon)
        gradient[idx] += diff
    return gradient

### Class Linear Gaussian Policy

In [ ]:
class LinearGaussianPolicy(object):
    """
    Time-varying linear Gaussian policy.
    U = K*x + k + noise, where noise ~ N(0, chol_pol_covar)
    """
    # TODO: Add additional noise patterns
    def __init__(self, K, k, pol_covar, chol_pol_covar, inv_pol_covar, check_shape, verbose=True):
        self.verbose = verbose
        if self.verbose: 
            print('-'*25)
            print('Call to the class LinearGaussianPolicy has started')
            print('-'*25)
        
        # Assume K has the correct shape, and make sure others match.
        self.T = K.shape[0]
        self.dU = K.shape[1]
        self.dX = K.shape[2]

        check_shape(k, (self.T, self.dU))
        check_shape(pol_covar, (self.T, self.dU, self.dU))
        check_shape(chol_pol_covar, (self.T, self.dU, self.dU))
        check_shape(inv_pol_covar, (self.T, self.dU, self.dU))

        self.K = K
        self.k = k
        self.pol_covar = pol_covar
        self.chol_pol_covar = chol_pol_covar
        self.inv_pol_covar = inv_pol_covar

    def act(self, x, obs, t, noise=None):
        """
        Return an action for a state.
        Args:
            x: State vector.
            obs: Observation vector.
            t: Time step.
            noise: Action noise. This will be scaled by the variance.
        """
        u = self.K[t].dot(x) + self.k[t]
        u += self.chol_pol_covar[t].T.dot(noise)
        return u

    def fold_k(self, noise):
        """
        Fold noise into k.
        Args:
            noise: A T x dU noise vector with mean 0 and variance 1.
        Returns:
            k: A T x dU bias vector.
        """
        k = np.zeros_like(self.k)
        for i in range(self.T):
            scaled_noise = self.chol_pol_covar[i].T.dot(noise[i])
            k[i] = scaled_noise + self.k[i]
        return k

    def nans_like(self):
        """
        Returns:
            A new linear Gaussian policy object with the same dimensions
            but all values filled with NaNs.
        """
        policy = LinearGaussianPolicy(
            np.zeros_like(self.K), np.zeros_like(self.k),
            np.zeros_like(self.pol_covar), np.zeros_like(self.chol_pol_covar),
            np.zeros_like(self.inv_pol_covar)
        )
        policy.K.fill(np.nan)
        policy.k.fill(np.nan)
        policy.pol_covar.fill(np.nan)
        policy.chol_pol_covar.fill(np.nan)
        policy.inv_pol_covar.fill(np.nan)
        return policy

### Class Trajectory Optimization Utilities 

In [ ]:
# TODO: Understand the part of how KL-Divergence are being computed.
class traj_opt_utils(object):
    '''
    init: defines some important parameters used by this function
    '''
    def __init__(self, DGD_MAX_ITER=50, DGD_MAX_LS_ITER=20, DGD_MAX_GD_ITER=200, 
                 ALPHA=0.005, BETA1=0.9, BETA2=0.999, EPS=1e-8, verbose=True):
        self.verbose = verbose
        if self.verbose:
            print('-'*25)
            print('Call to the class traj_opt_utils has started')
            print('-'*25)
        # TODO: Maybe add this parameter to TrajOptLQR
        # Constants used in TrajOptLQR.
        self.DGD_MAX_ITER = 50
        self.DGD_MAX_LS_ITER = 20
        self.DGD_MAX_GD_ITER = 200
        # Adam parameters
        self.ALPHA, self.BETA1, self.BETA2, self.EPS = 0.005, 0.9, 0.999, 1e-8  
        
    def traj_distr_kl(self, new_mu, new_sigma, new_traj_distr, prev_traj_distr, tot=True):
    """
    Compute KL divergence between new and previous trajectory
    distributions.
    Args:
        new_mu: T x dX, mean of new trajectory distribution.
        new_sigma: T x dX x dX, variance of new trajectory distribution.
        new_traj_distr: A linear Gaussian policy object, new
            distribution.
        prev_traj_distr: A linear Gaussian policy object, previous
            distribution.
        tot: Whether or not to sum KL across all time steps.
    Returns:
        kl_div: The KL divergence between the new and previous
            trajectories.
    """
    # Constants.
    T = new_mu.shape[0]
    dU = new_traj_distr.dU

    # Initialize vector of divergences for each time step.
    kl_div = np.zeros(T)

    # Step through trajectory.
    for t in range(T):
        # Fetch matrices and vectors from trajectory distributions.
        mu_t = new_mu[t, :]
        sigma_t = new_sigma[t, :, :]
        K_prev = prev_traj_distr.K[t, :, :]
        K_new = new_traj_distr.K[t, :, :]
        k_prev = prev_traj_distr.k[t, :]
        k_new = new_traj_distr.k[t, :]
        chol_prev = prev_traj_distr.chol_pol_covar[t, :, :]
        chol_new = new_traj_distr.chol_pol_covar[t, :, :]

        # Compute log determinants and precision matrices.
        logdet_prev = 2 * sum(np.log(np.diag(chol_prev)))
        logdet_new = 2 * sum(np.log(np.diag(chol_new)))
        prc_prev = sp.linalg.solve_triangular(
            chol_prev, sp.linalg.solve_triangular(chol_prev.T, np.eye(dU),
                                                  lower=True)
        )
        prc_new = sp.linalg.solve_triangular(
            chol_new, sp.linalg.solve_triangular(chol_new.T, np.eye(dU),
                                                 lower=True)
        )

        # Construct matrix, vector, and constants.
        M_prev = np.r_[
            np.c_[K_prev.T.dot(prc_prev).dot(K_prev), -K_prev.T.dot(prc_prev)],
            np.c_[-prc_prev.dot(K_prev), prc_prev]
        ]
        M_new = np.r_[
            np.c_[K_new.T.dot(prc_new).dot(K_new), -K_new.T.dot(prc_new)],
            np.c_[-prc_new.dot(K_new), prc_new]
        ]
        v_prev = np.r_[K_prev.T.dot(prc_prev).dot(k_prev),
                       -prc_prev.dot(k_prev)]
        v_new = np.r_[K_new.T.dot(prc_new).dot(k_new), -prc_new.dot(k_new)]
        c_prev = 0.5 * k_prev.T.dot(prc_prev).dot(k_prev)
        c_new = 0.5 * k_new.T.dot(prc_new).dot(k_new)

        # Compute KL divergence at timestep t.
        kl_div[t] = max(
            0,
            -0.5 * mu_t.T.dot(M_new - M_prev).dot(mu_t) -
            mu_t.T.dot(v_new - v_prev) - c_new + c_prev -
            0.5 * np.sum(sigma_t * (M_new-M_prev)) - 0.5 * logdet_new +
            0.5 * logdet_prev
        )

    # Add up divergences across time to get total divergence.
    return np.sum(kl_div) if tot else kl_div


    def traj_distr_kl_alt(self, new_mu, new_sigma, new_traj_distr, prev_traj_distr, tot=True):
    """
    This function computes the same quantity as the function above.
    However, it is easier to modify and understand this function, i.e.,
    passing in a different mu and sigma to this function will behave properly.
    """
    T, dX, dU = new_mu.shape[0], new_traj_distr.dX, new_traj_distr.dU
    kl_div = np.zeros(T)

    for t in range(T):
        K_prev = prev_traj_distr.K[t, :, :]
        K_new = new_traj_distr.K[t, :, :]

        k_prev = prev_traj_distr.k[t, :]
        k_new = new_traj_distr.k[t, :]

        sig_prev = prev_traj_distr.pol_covar[t, :, :]
        sig_new = new_traj_distr.pol_covar[t, :, :]

        chol_prev = prev_traj_distr.chol_pol_covar[t, :, :]
        chol_new = new_traj_distr.chol_pol_covar[t, :, :]

        inv_prev = prev_traj_distr.inv_pol_covar[t, :, :]
        inv_new = new_traj_distr.inv_pol_covar[t, :, :]

        logdet_prev = 2 * sum(np.log(np.diag(chol_prev)))
        logdet_new = 2 * sum(np.log(np.diag(chol_new)))

        K_diff, k_diff = K_prev - K_new, k_prev - k_new
        mu, sigma = new_mu[t, :dX], new_sigma[t, :dX, :dX]

        kl_div[t] = max(
                0,
                0.5 * (logdet_prev - logdet_new - new_traj_distr.dU +
                       np.sum(np.diag(inv_prev.dot(sig_new))) +
                       k_diff.T.dot(inv_prev).dot(k_diff) +
                       mu.T.dot(K_diff.T).dot(inv_prev).dot(K_diff).dot(mu) +
                       np.sum(np.diag(K_diff.T.dot(inv_prev).dot(K_diff).dot(sigma))) +
                       2 * k_diff.T.dot(inv_prev).dot(K_diff).dot(mu))
        )

    return np.sum(kl_div) if tot else kl_div


    def approximated_cost(self, sample_list, traj_distr, traj_info):
    """
    This function gives the LQR estimate of the cost function given the noise
    experienced along each sample in sample_list.
    Args:
        sample_list: List of samples to extract noise from.
        traj_distr: LQR controller to roll forward.
        traj_info: Used to obtain dynamics estimate to simulate trajectories.
    Returns:
        mu_all: Trajectory means corresponding to each sample in sample_list.
        predicted_cost: LQR estimates of cost of each sample in sample_list.
    """
    T = traj_distr.T
    N = len(sample_list)
    dU = traj_distr.dU
    dX = traj_distr.dX
    noise = sample_list.get_noise()

    # Constants.
    idx_x = slice(dX)
    mu_all = np.zeros((N, T, dX+dU))

    # Pull out dynamics.
    Fm = traj_info.dynamics.Fm
    fv = traj_info.dynamics.fv
    dyn_covar = traj_info.dynamics.dyn_covar

    for i in range(N):
        mu = np.zeros((T, dX+dU))
        mu[0, idx_x] = traj_info.x0mu
        for t in range(T):
            mu[t, :] = np.hstack([
                mu[t, idx_x],
                (traj_distr.K[t, :, :].dot(mu[t, idx_x]) + traj_distr.k[t, :]
                 + traj_distr.chol_pol_covar[t].T.dot(noise[i, t]))
            ])
            if t < T - 1:
                mu[t+1, idx_x] = Fm[t, :, :].dot(mu[t, :]) + fv[t, :]
        mu_all[i, :, :] = mu

    # Compute cost.
    predicted_cost = np.zeros((N, T))

    for i in range(N):
        for t in range(T):
            predicted_cost[i, t] = traj_info.cc[t] + \
                    0.5 * mu_all[i,t,:].T.dot(traj_info.Cm[t, :, :]).dot(mu_all[i,t,:]) + \
                    mu_all[i,t,:].T.dot(traj_info.cv[t, :])

    return mu_all, predicted_cost
        

## File: Class Bundle which is a super class and all other depending classes 

In [ ]:
'''
Add the following line because PolicyInfo class needs it:
    from ??? import LinearGaussianPolicy
or modify it by sending it as an argument but then either that class needs to be in same file or 
imported from other file. If you end up doing chosing latter then follow above methods that is better.
'''

class BundleType(object):
    """
    This class bundles many fields, similar to a record or a mutable
    namedtuple. 
        Which means thatif the subclass is initiated with this init function then 
        all the parameters in dictionary that are fed through variables will be made
        into self. parameters of that subclass
    """
    def __init__(self, variables):
        for var, val in variables.items():
            object.__setattr__(self, var, val)

    # Freeze fields so new ones cannot be set.
    def __setattr__(self, key, value):
        if not hasattr(self, key):
            raise AttributeError("%r has no attribute %s" % (self, key))
        object.__setattr__(self, key, value)
        
class IterationData(BundleType):
    """ Collection of iteration variables. """
    def __init__(self):
        variables = {
            'sample_list': None,    # List of samples for the current iteration.
            'traj_info': None,      # Current TrajectoryInfo object.
            'pol_info': None,       # Current PolicyInfo object.
            'traj_distr': None,     # Initial trajectory distribution.
            'new_traj_distr': None, # Updated trajectory distribution.
            'cs': None,             # Sample costs of the current iteration.
            'step_mult': 1.0,       # KL step multiplier for the current iteration.
            'eta': 1.0,             # Dual variable used in LQR backward pass.
        }
        BundleType.__init__(self, variables)


class TrajectoryInfo(BundleType):
    """ Collection of trajectory-related variables. """
    def __init__(self):
        variables = {
            'dynamics': None,              # Dynamics object for the current iteration.
            'x0mu': None,                  # Mean for the initial state, used by the dynamics.
            'x0sigma': None,               # Covariance for the initial state distribution.
            'cc': None,                    # Cost estimate constant term.
            'cv': None,                    # Cost estimate vector term.
            'Cm': None,                    # Cost estimate matrix term.
            'last_kl_step': float('inf'),  # KL step of the previous iteration.
        }
        BundleType.__init__(self, variables)
        
class PolicyInfo(BundleType):
    """ Collection of policy-related variables. """
    def __init__(self, hyperparams):
        T, dU, dX = hyperparams['T'], hyperparams['dU'], hyperparams['dX']
        variables = {
            'lambda_k': np.zeros((T, dU)),                      # Dual variables.
            'lambda_K': np.zeros((T, dU, dX)),                  # Dual variables.
            'pol_wt': hyperparams['init_pol_wt'] * np.ones(T),  # Policy weight.
            'pol_mu': None,                       # Mean of the current policy output.
            'pol_sig': None,                      # Covariance of the current policy output.
            'pol_K': np.zeros((T, dU, dX)),       # Policy linearization.
            'pol_k': np.zeros((T, dU)),           # Policy linearization.
            'pol_S': np.zeros((T, dU, dU)),       # Policy linearization covariance.
            'chol_pol_S': np.zeros((T, dU, dU)),  # Cholesky decomp of covar.
            'prev_kl': None,                      # Previous KL divergence.
            'init_kl': None,                      # The initial KL divergence, before the iteration.
            'policy_samples': [],                 # List of current policy samples.
            'policy_prior': None,                 # Current prior for policy linearization.
        }
        BundleType.__init__(self, variables)

    def traj_distr(self):
        """ Create a trajectory distribution object from policy info. """
        T, dU, dX = self.pol_K.shape
        # Compute inverse policy covariances.
        inv_pol_S = np.empty_like(self.chol_pol_S)
        for t in range(T):
            inv_pol_S[t, :, :] = np.linalg.solve(
                self.chol_pol_S[t, :, :],
                np.linalg.solve(self.chol_pol_S[t, :, :].T, np.eye(dU))
            )
        return LinearGaussianPolicy(self.pol_K, self.pol_k, self.pol_S,
                self.chol_pol_S, inv_pol_S)

In [ ]:
class algorithm_utils(object):
    '''
    Init function not required. Currently there for debugging.
        init: define the class constructor of with no arguments
    '''
    def __init__(self, verbose=True):
        self.verbose = verbose
        if self.verbose:
            print('-'*25)
            print('Call to the class algorithm_utils has started')
            print('-'*25)

    def estimate_moments(self, X, mu, covar):
        """ Estimate the moments for a given linearized policy. """
        N, T, dX = X.shape
        dU = mu.shape[-1]
        if len(covar.shape) == 3:
            covar = np.tile(covar, [N, 1, 1, 1])
        Xmu = np.concatenate([X, mu], axis=2)
        ev = np.mean(Xmu, axis=0)
        em = np.zeros((N, T, dX+dU, dX+dU))
        pad1 = np.zeros((dX, dX+dU))
        pad2 = np.zeros((dU, dX))
        for n in range(N):
            for t in range(T):
                covar_pad = np.vstack([pad1, np.hstack([pad2, covar[n, t, :, :]])])
                em[n, t, :, :] = np.outer(Xmu[n, t, :], Xmu[n, t, :]) + covar_pad
        return ev, em


    def gauss_fit_joint_prior(self, pts, mu0, Phi, m, n0, dwts, dX, dU, sig_reg):
        """ Perform Gaussian fit to data with a prior. """
        # Build weights matrix.
        D = np.diag(dwts)
        # Compute empirical mean and covariance.
        mun = np.sum((pts.T * dwts).T, axis=0)
        diff = pts - mun
        empsig = diff.T.dot(D).dot(diff)
        empsig = 0.5 * (empsig + empsig.T)
        # MAP estimate of joint distribution.
        N = dwts.shape[0]
        mu = mun
        sigma = (N * empsig + Phi + (N * m) / (N + m) *
                 np.outer(mun - mu0, mun - mu0)) / (N + n0)
        sigma = 0.5 * (sigma + sigma.T)
        # Add sigma regularization.
        sigma += sig_reg
        # Conditioning to get dynamics.
        fd = np.linalg.solve(sigma[:dX, :dX], sigma[:dX, dX:dX+dU]).T
        fc = mu[dX:dX+dU] - fd.dot(mu[:dX])
        dynsig = sigma[dX:dX+dU, dX:dX+dU] - fd.dot(sigma[:dX, :dX]).dot(fd.T)
        dynsig = 0.5 * (dynsig + dynsig.T)
        return fd, fc, dynsig

## Cost Utilities

In [ ]:
class cost_utils(object):
    '''
    init: start the function with some important parameters
    '''
    def __init__(self):
        self.RAMP_CONSTANT = 1
        self.RAMP_LINEAR = 2
        self.RAMP_QUADRATIC = 3
        self.RAMP_FINAL_ONLY = 4
        
    def get_ramp_multiplier(self, ramp_option, T, wp_final_multiplier=1.0):
    """
    Return a time-varying multiplier.
    Returns:
        A (T,) float vector containing weights for each time step.
    """
    if ramp_option == self.RAMP_CONSTANT:
        wpm = np.ones(T)
    elif ramp_option == self.RAMP_LINEAR:
        wpm = (np.arange(T, dtype=np.float32) + 1) / T
    elif ramp_option == self.RAMP_QUADRATIC:
        wpm = ((np.arange(T, dtype=np.float32) + 1) / T) ** 2
    elif ramp_option == self.RAMP_FINAL_ONLY:
        wpm = np.zeros(T)
        wpm[T-1] = 1.0
    else:
        raise ValueError('Unknown cost ramp requested!')
    wpm[-1] *= wp_final_multiplier
    return wpm


    def evall1l2term(self, wp, d, Jd, Jdd, l1, l2, alpha):
    """
    Evaluate and compute derivatives for combined l1/l2 norm penalty.
    loss = (0.5 * l2 * d^2) + (l1 * sqrt(alpha + d^2))
    Args:
        wp: T x D matrix with weights for each dimension and time step.
        d: T x D states to evaluate norm on.
        Jd: T x D x Dx Jacobian - derivative of d with respect to state.
        Jdd: T x D x Dx x Dx Jacobian - 2nd derivative of d with respect
            to state.
        l1: l1 loss weight.
        l2: l2 loss weight.
        alpha: Constant added in square root.
    """
    # Get trajectory length.
    T, _ = d.shape

    # Compute scaled quantities.
    sqrtwp = np.sqrt(wp)
    dsclsq = d * sqrtwp
    dscl = d * wp
    dscls = d * (wp ** 2)

    # Compute total cost.
    l = 0.5 * np.sum(dsclsq ** 2, axis=1) * l2 + \
            np.sqrt(alpha + np.sum(dscl ** 2, axis=1)) * l1

    # First order derivative terms.
    d1 = dscl * l2 + (
        dscls / np.sqrt(alpha + np.sum(dscl ** 2, axis=1, keepdims=True)) * l1
    )
    lx = np.sum(Jd * np.expand_dims(d1, axis=2), axis=1)

    # Second order terms.
    psq = np.expand_dims(
        np.sqrt(alpha + np.sum(dscl ** 2, axis=1, keepdims=True)), axis=1
    )
    d2 = l1 * (
        (np.expand_dims(np.eye(wp.shape[1]), axis=0) *
         (np.expand_dims(wp ** 2, axis=1) / psq)) -
        ((np.expand_dims(dscls, axis=1) *
          np.expand_dims(dscls, axis=2)) / psq ** 3)
    )
    d2 += l2 * (
        np.expand_dims(wp, axis=2) * np.tile(np.eye(wp.shape[1]), [T, 1, 1])
    )

    d1_expand = np.expand_dims(np.expand_dims(d1, axis=-1), axis=-1)
    sec = np.sum(d1_expand * Jdd, axis=1)

    Jd_expand_1 = np.expand_dims(np.expand_dims(Jd, axis=2), axis=4)
    Jd_expand_2 = np.expand_dims(np.expand_dims(Jd, axis=1), axis=3)
    d2_expand = np.expand_dims(np.expand_dims(d2, axis=-1), axis=-1)
    lxx = np.sum(np.sum(Jd_expand_1 * Jd_expand_2 * d2_expand, axis=1), axis=1)

    lxx += 0.5 * sec + 0.5 * np.transpose(sec, [0, 2, 1])

    return l, lx, lxx


    def evallogl2term(self, wp, d, Jd, Jdd, l1, l2, alpha):
    """
    Evaluate and compute derivatives for combined l1/l2 norm penalty.
    loss = (0.5 * l2 * d^2) + (0.5 * l1 * log(alpha + d^2))
    Args:
        wp: T x D matrix with weights for each dimension and time step.
        d: T x D states to evaluate norm on.
        Jd: T x D x Dx Jacobian - derivative of d with respect to state.
        Jdd: T x D x Dx x Dx Jacobian - 2nd derivative of d with respect
            to state.
        l1: l1 loss weight.
        l2: l2 loss weight.
        alpha: Constant added in square root.
    """
    # Get trajectory length.
    T, _ = d.shape

    # Compute scaled quantities.
    sqrtwp = np.sqrt(wp)
    dsclsq = d * sqrtwp
    dscl = d * wp
    dscls = d * (wp ** 2)

    # Compute total cost.
    l = 0.5 * np.sum(dsclsq ** 2, axis=1) * l2 + \
            0.5 * np.log(alpha + np.sum(dscl ** 2, axis=1)) * l1
    # First order derivative terms.
    d1 = dscl * l2 + (
        dscls / (alpha + np.sum(dscl ** 2, axis=1, keepdims=True)) * l1
    )
    lx = np.sum(Jd * np.expand_dims(d1, axis=2), axis=1)

    # Second order terms.
    psq = np.expand_dims(
        alpha + np.sum(dscl ** 2, axis=1, keepdims=True), axis=1
    )
    #TODO: Need * 2.0 somewhere in following line, or * 0.0 which is
    #      wrong but better.
    d2 = l1 * (
        (np.expand_dims(np.eye(wp.shape[1]), axis=0) *
         (np.expand_dims(wp ** 2, axis=1) / psq)) -
        ((np.expand_dims(dscls, axis=1) *
          np.expand_dims(dscls, axis=2)) / psq ** 2)
    )
    d2 += l2 * (
        np.expand_dims(wp, axis=2) * np.tile(np.eye(wp.shape[1]), [T, 1, 1])
    )

    d1_expand = np.expand_dims(np.expand_dims(d1, axis=-1), axis=-1)
    sec = np.sum(d1_expand * Jdd, axis=1)

    Jd_expand_1 = np.expand_dims(np.expand_dims(Jd, axis=2), axis=4)
    Jd_expand_2 = np.expand_dims(np.expand_dims(Jd, axis=1), axis=3)
    d2_expand = np.expand_dims(np.expand_dims(d2, axis=-1), axis=-1)
    lxx = np.sum(np.sum(Jd_expand_1 * Jd_expand_2 * d2_expand, axis=1), axis=1)

    lxx += 0.5 * sec + 0.5 * np.transpose(sec, [0, 2, 1])

    return l, lx, lxx 

## Cost function are ready but needs slight modification based on Hyperparam.py file

In [ ]:
# class CostAction(Cost):
#     """ 
#     This class defines the torque (action) cost. 
#     Computes torque penalties.
#     """
#     def __init__(self, hyperparams, COST_ACTION):
#         config = deepcopy(COST_ACTION)
#         config.update(hyperparams)
#         self._hyperparams = config

#     def eval(self, x, u):
#         """
#         Evaluate cost function and derivatives on a sample.
#         Args:
#             sample: A single sample in the following form X[T, Dx] and U[T, Du].
#         """
#         T = x.shape[0]
#         Du = u.shape[1]
#         Dx = x.shape[1]
#         l = 0.5 * np.sum(self._hyperparams['wu'] * (u ** 2), axis=1)
#         lu = self._hyperparams['wu'] * sample_u
#         lx = np.zeros((T, Dx))
#         luu = np.tile(np.diag(self._hyperparams['wu']), [T, 1, 1])
#         lxx = np.zeros((T, Dx, Dx))
#         lux = np.zeros((T, Du, Dx))
#         return l, lx, lu, lxx, luu, lux

In [ ]:
# class CostSum(object):
#     """ 
#     This class defines a cost sum of arbitrary other costs. A wrapper cost 
#     function that adds other cost functions. 
#     """
#     def __init__(self, hyperparams, COST_SUM):
#         config = deepcopy(COST_SUM)
#         config.update(hyperparams)
#         self._hyperparams = config

#         self._costs = []
#         self._weights = self._hyperparams['weights']

#         for cost in self._hyperparams['costs']:
#             self._costs.append(cost['type'](cost))
#             # here the two different cost i.e action cost and the state cost is calculated

#     def eval(self, x, u):
#         """
#         Evaluate cost function and derivatives.
#         Args:
#             sample: A single sample in the following form X[T, Dx] and U[T, Du].
#         """
#         l, lx, lu, lxx, luu, lux = self._costs[0].eval(sample)
#         # getting cost from each cost function and storing in the variable

#         # Compute weighted sum of each cost value and derivatives.
#         weight = self._weights[0]
#         l = l * weight
#         lx = lx * weight
#         lu = lu * weight
#         lxx = lxx * weight
#         luu = luu * weight
#         lux = lux * weight
#         for i in range(1, len(self._costs)):
#             pl, plx, plu, plxx, pluu, plux = self._costs[i].eval(sample)
#             weight = self._weights[i]
#             l = l + pl * weight
#             lx = lx + plx * weight
#             lu = lu + plu * weight
#             lxx = lxx + plxx * weight
#             luu = luu + pluu * weight
#             lux = lux + plux * weight
#         return l, lx, lu, lxx, luu, lux

In [ ]:
# class CostState(object):
#     """ 
#     This file defines the state target cost. 
#     Computes l1/l2 distance to a fixed target state. 
#     """
#     def __init__(self, hyperparams, COST_STATE, evall1l2term, get_ramp_multiplier):
#         config = copy.deepcopy(COST_STATE)
#         config.update(hyperparams)
#         self._hyperparams = config
#         self.evall1l2term = evall1l2term
#         self.get_ramp_multiplier = get_ramp_multiplier

#     def eval(self, x, u):
#         """
#         Evaluate cost function and derivatives on a sample.
#         Args:
#             sample: A single sample in the following form X[T, Dx] and U[T, Du].
#         """
#         T = x.shape[0]
#         Du = u.shape[1]
#         Dx = x.shape[1]

#         final_l = np.zeros(T)
#         final_lu = np.zeros((T, Du))
#         final_lx = np.zeros((T, Dx))
#         final_luu = np.zeros((T, Du, Du))
#         final_lxx = np.zeros((T, Dx, Dx))
#         final_lux = np.zeros((T, Du, Dx))

#         for data_type in self._hyperparams['data_types']:
#             config = self._hyperparams['data_types'][data_type]
#             wp = config['wp']
#             tgt = config['target_state']
#             # TODO: This may fail.
#             _, dim_sensor = x.shape

#             wpm = self.get_ramp_multiplier(
#                 self._hyperparams['ramp_option'], T,
#                 wp_final_multiplier=self._hyperparams['wp_final_multiplier']
#             )
#             wp = wp * np.expand_dims(wpm, axis=-1)
#             # Compute state penalty.
#             dist = x - tgt

#             # Evaluate penalty term.
#             l, ls, lss = self.evall1l2term(
#                 wp, dist, np.tile(np.eye(dim_sensor), [T, 1, 1]),
#                 np.zeros((T, dim_sensor, dim_sensor, dim_sensor)),
#                 self._hyperparams['l1'], self._hyperparams['l2'],
#                 self._hyperparams['alpha']
#             )

#             final_l += l

#             # sample.agent.pack_data_x(final_lx, ls, data_types=[data_type])
#             # sample.agent.pack_data_x(final_lxx, lss, data_types=[data_type, data_type])
#         return final_l, final_lx, final_lu, final_lxx, final_luu, final_lux

## Hyperparameters file
## Need more modification not READY YET

In [ ]:
""" 
Hyperparameters for Hopper Problem.
Note:
    1 - All the angles are in degrees. 
"""

import os
from datetime import datetime
import numpy as np

# from gps import __file__ as gps_filepath
# from gps.agent.box2d.agent_box2d import AgentBox2D
# from gps.agent.box2d.arm_world import ArmWorld
# from gps.algorithm.algorithm_badmm import AlgorithmBADMM
# from gps.algorithm.cost.cost_state import CostState
# from gps.algorithm.cost.cost_action import CostAction
# from gps.algorithm.cost.cost_sum import CostSum
# from gps.algorithm.dynamics.dynamics_lr_prior import DynamicsLRPrior
# from gps.algorithm.dynamics.dynamics_prior_gmm import DynamicsPriorGMM
# from gps.algorithm.policy.policy_prior_gmm import PolicyPriorGMM
# from gps.algorithm.traj_opt.traj_opt_lqr_python import TrajOptLQRPython
# from gps.algorithm.policy_opt.policy_opt_caffe import PolicyOptCaffe
# from gps.algorithm.policy.lin_gauss_init import init_lqr
# from gps.gui.config import generate_experiment_info
# from gps.proto.gps_pb2 import JOINT_ANGLES, JOINT_VELOCITIES, END_EFFECTOR_POINTS, ACTION

SENSOR_DIMS = {
    POS: 2,
    ANGLES: 4,
    X: 6,
    U: 3,
}

BASE_DIR = 'checkpoints_GPS/'
EXP_DIR = BASE_DIR + '/Logs'


common = {
    'experiment_name': 'CartPole' + '_' + datetime.strftime(datetime.now(), '%m-%d-%y_%H-%M'),
    'experiment_dir': EXP_DIR,
    'data_files_dir': EXP_DIR + 'data_files/',
    'log_filename': EXP_DIR + 'log.txt',
    'conditions': 4, # TODO: What is this param? Maybe rollouts
}

if not os.path.exists(common['data_files_dir']):
    os.makedirs(common['data_files_dir'])

# X: [ZPos_Torso, XPos_Torso, YPos_Torso, Theta_Thigh, Theta_Leg, Theta_Foot]
agent = {
    'type': Hopper_Sim,
#     'target_state' : np.array([0, 0, 0, 0]),
#     'x0': [np.array([0.5*np.pi, 0, 0, 0, 0, 0, 0]),
#            np.array([0.75*np.pi, 0.5*np.pi, 0, 0, 0, 0, 0]),
#            np.array([np.pi, -0.5*np.pi, 0, 0, 0, 0, 0]),
#            np.array([1.25*np.pi, 0, 0, 0, 0, 0, 0]),
#           ],
    'rk': 0,
    'dt': 0.05,
    'substeps': 1,
    'conditions': common['conditions'],
#     'pos_body_idx': np.array([]),
#     'pos_body_offset': np.array([]),
    'T': 100,
    'sensor_dims': SENSOR_DIMS,
    'state_include': [JOINT_ANGLES, JOINT_VELOCITIES, END_EFFECTOR_POINTS],
    'obs_include': [JOINT_ANGLES, JOINT_VELOCITIES, END_EFFECTOR_POINTS],
}

algorithm = {
    'type': AlgorithmBADMM,
    'conditions': common['conditions'],
    'iterations': 10,
    'lg_step_schedule': np.array([1e-4, 1e-3, 1e-2, 1e-2]),
    'policy_dual_rate': 0.2,
    'ent_reg_schedule': np.array([1e-3, 1e-3, 1e-2, 1e-1]),
    'fixed_lg_step': 3,
    'kl_step': 5.0,
    'min_step_mult': 0.01,
    'max_step_mult': 1.0,
    'sample_decrease_var': 0.05,
    'sample_increase_var': 0.1,
}

algorithm['init_traj_distr'] = {
    'type': init_lqr,
    'init_gains': np.zeros(SENSOR_DIMS[ACTION]),
    'init_acc': np.zeros(SENSOR_DIMS[ACTION]),
    'init_var': 0.1,
    'stiffness': 0.01,
    'dt': agent['dt'],
    'T': agent['T'],
}

action_cost = {
    'type': CostAction,
    'wu': np.array([1, 1])
}

state_cost = {
    'type': CostState,
    'data_types' : {
        JOINT_ANGLES: {
            'wp': np.array([1, 1]),
            'target_state': agent["target_state"],
        },
    },
}

algorithm['cost'] = {
    'type': CostSum,
    'costs': [action_cost, state_cost],
    'weights': [1e-5, 1.0],
}

algorithm['dynamics'] = {
    'type': DynamicsLRPrior,
    'regularization': 1e-6,
    'prior': {
        'type': DynamicsPriorGMM,
        'max_clusters': 20,
        'min_samples_per_cluster': 40,
        'max_samples': 20,
    },
}

algorithm['traj_opt'] = {
    'type': TrajOptLQRPython,
}

algorithm['policy_opt'] = {
    'type': PolicyOptCaffe,
    'weights_file_prefix': EXP_DIR + 'policy',
}

algorithm['policy_prior'] = {
    'type': PolicyPriorGMM,
    'max_clusters': 20,
    'min_samples_per_cluster': 40,
    'max_samples': 20,
}

config = {
    'iterations': 10,
    'num_samples': 5,
    'verbose_trials': 5,
    'verbose_policy_trials': 0,
    'common': common,
    'agent': agent,
    'gui_on': True,
    'algorithm': algorithm,
}

common['info'] = generate_experiment_info(config)


## Scrap code

In [31]:
# def extract_condition(hyperparams, m):
#     """
#     Pull the relevant hyperparameters corresponding to the specified
#     condition, and return a new hyperparameter dictionary.
#     """
#     return {var: val[m] if isinstance(val, list) else val for var, val in hyperparams.items()}

# algorithm = {
#     'type': 'init_lqr',
#     'init_gains': np.zeros(6),
#     'init_acc': np.zeros(6),
#     'init_var': 0.1,
#     'stiffness': 0.01,
#     'dt': 1,
#     'T': 2,
# }
# INIT_LG_LQR = {
#     'wow': algorithm,
#     'init_var': 1,
#     'stiffness': 2,
#     'stiffness_vel': 3,
#     'final_weight': 4,
#     # Parameters for guessing dynamics
#     # dU vector of accelerations, default zeros.
#     'init_acc': [],  
#     # dU vector of gains, default ones.
#     'init_gains': [],  
# }


# temp = range(10)
# b = extract_condition(algorithm, temp)
# # b = extract_condition(INIT_LG_LQR, temp[0])
# print(b)


# for var, val in INIT_LG_LQR.items():
#     if isinstance(val, list):
#         print('in if')
#         a = {var: val[1]}
#         print(a)
#     else:
#         print('in else')
#         print({val})

In [9]:
a = np.array([[1, 2], [3, 4]])
b = np.array([a, a*2])
c = b*3
print(np.sum(b, axis=0))
print('mean\n', np.sum(b, axis=0)/b.shape[0])

[[ 3  6]
 [ 9 12]]
mean
 [[1.5 3. ]
 [4.5 6. ]]


In [11]:
from scipy.stats.mstats import gmean
print(b[0]*b[1])
print(gmean(b, axis=0))

[[ 2  8]
 [18 32]]
[[1.41421356 2.82842712]
 [4.24264069 5.65685425]]


In [3]:
        try:
            assert len([1,2]) != 1
        except:
            raise AssertionError('condition is one please change its!!!!1')

In [8]:
import numpy as np
len(np.zeros((12,6)))

12

In [15]:
X = np.zeros((5,6))

In [13]:
X.fill(np.nan)

In [16]:
t = 2
X[t,:].fill(np.nan)

In [26]:
INIT_LG_LQR = {
    'wow': 'algorithm',
    'init_var': 1,
    'stiffness': 2,
    'stiffness_vel': 3,
    'final_weight': 4,
    # Parameters for guessing dynamics
    # dU vector of accelerations, default zeros.
    'init_acc': [],  
    # dU vector of gains, default ones.
    'init_gains': [],  
}

In [28]:
for i in INIT_LG_LQR:
    print(i)
    print(len(i))

wow
3
init_var
8
stiffness
9
stiffness_vel
13
final_weight
12
init_acc
8
init_gains
10
